In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
import os
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from sklearn.metrics import matthews_corrcoef
import csv

In [16]:
from fastai import *
from fastai.vision import *

In [17]:
from data_loader import ImageItemListCell

In [18]:
path = Path('/data/Datasets/WhiteBloodCancer/train/')
path_submission = Path('/data/Datasets/WhiteBloodCancer/train/submission/')
path_models = Path('/data/Datasets/WhiteBloodCancer/train/ensemble/')
test_path = Path('/data/Datasets/WhiteBloodCancer/test/')

test_files = get_files(test_path)
test = ImageItemListCell(test_files)

test_files[:10]

[PosixPath('/data/Datasets/WhiteBloodCancer/test/1066.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/710.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/1384.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/996.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/1604.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/232.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/1302.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/628.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/1467.bmp'),
 PosixPath('/data/Datasets/WhiteBloodCancer/test/905.bmp')]

In [12]:
for model in path_models.glob('*.pkl'):

    learn = load_learner(path, model, test)
    learn.model.float()
    
    # Use TTA
    tta_result, y_test_tta = learn.TTA(ds_type=DatasetType.Test, beta=0.6, scale=1.3)
    y_pred = np.argmax(tta_result, axis=1)
    
    # use normal prediction
    #preds_test,y_test=learn.get_preds(ds_type=DatasetType.Test)
    #y_pred = np.argmax(torch.sigmoid(preds_test), axis=1)
    
    with open(path_submission.joinpath('{}.csv'.format(model.stem)), mode='w') as submission:
        submission_writer = csv.writer(submission, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

        for fn, y in zip(learn.data.test_dl.items, to_np(y_pred)):
            submission_writer.writerow([fn.name, y]) #learn.data.classes[y]

In [21]:
ensemble = np.zeros(shape=(len(learn.data.test_dl.items)))
model_count = len(list(path_submission.glob('*.csv')))
for submission in path_submission.glob('*.csv'):
    
    with open(submission, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for i, row in enumerate(reader):
            ensemble[i] += int(row[1])

result = np.rint(ensemble / model_count).astype(int)
submission = ["" for i in range(ensemble.shape[0])]

for fn, y in zip(learn.data.test_dl.items, result):
    index = int(fn.name.replace(".bmp",'')) - 1
    #submission[index] = "{},{}".format(fn.name, learn.data.classes[y])
    y = 1 if y == 0 else 0
    submission[index] = "{}".format(y)

with open(path_submission.joinpath('isbi_valid.predict'), 'w') as f:
    for item in submission:
        f.write("{}\n".format(item))